# Task3

Index_X = FSR_for_coord

Index_y = x_coord, y_coord

Data = Splited by Time

## Run result

https://wandb.ai/seokjin/FSR-prediction/groups/FSR_Trainable_2023-07-19_02-32-25/workspace?workspace=user-seokjin

## Experiment id

FSR_Trainable_2023-07-19_02-32-25

## Best metric (RMSE)

0.6551

In [1]:
def define_searchspace(trial):
    model = trial.suggest_categorical('model', ['fsr_model.ANN'])
    if model == 'fsr_model.LSTM':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    elif model == 'fsr_model.CNN_LSTM':
        trial.suggest_categorical('model_args/cnn_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_categorical('model_args/lstm_hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/cnn_num_layer', 1, 8)
        trial.suggest_int('model_args/lstm_num_layer', 1, 8)
    elif model == 'fsr_model.ANN':
        trial.suggest_categorical('model_args/hidden_size', [8, 16, 32, 64, 128])
        trial.suggest_int('model_args/num_layer', 1, 8)
    trial.suggest_categorical('criterion', ['torch.nn.MSELoss'])
    trial.suggest_categorical('optimizer', [
        'torch.optim.Adam',
        'torch.optim.NAdam',
        'torch.optim.Adagrad',
        'torch.optim.RAdam',
        'torch.optim.SGD',
    ])
    trial.suggest_float('optimizer_args/lr', 1e-5, 1e-1, log=True)
    imputer = trial.suggest_categorical('imputer', ['sklearn.impute.SimpleImputer'])
    if imputer == 'sklearn.impute.SimpleImputer':
        trial.suggest_categorical('imputer_args/strategy', [
            'mean',
            'median',
        ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    trial.suggest_categorical('scaler', [ 
        'sklearn.preprocessing.StandardScaler',
        'sklearn.preprocessing.MinMaxScaler',
        'sklearn.preprocessing.RobustScaler',
    ])
    return {
        'index_X': 'FSR_for_coord',
        'index_y': ['x_coord', 'y_coord'],
        'data_loader': 'fsr_data.get_index_splited_by_time'
    }

In [2]:
import ray.tune
import ray.air
import ray.air.integrations.wandb
import ray.tune.schedulers
from fsr_trainable import FSR_Trainable
import ray.tune.search
import ray.tune.search.optuna

tuner = ray.tune.Tuner(
    trainable=ray.tune.with_resources(
        FSR_Trainable, {'cpu':2},
    ),
    tune_config=ray.tune.TuneConfig(
        num_samples=100,
        scheduler=ray.tune.schedulers.ASHAScheduler(
            max_t=100,
            grace_period=1,
            reduction_factor=2,
            brackets=1,
            metric='rmse',
            mode='min',
        ),
        search_alg=ray.tune.search.optuna.OptunaSearch(
            space=define_searchspace,
            metric='rmse',
            mode='min',
        ),
    ), 
    run_config=ray.air.RunConfig(
        callbacks=[
            ray.air.integrations.wandb.WandbLoggerCallback(project='FSR-prediction'),
        ],
        checkpoint_config=ray.air.CheckpointConfig(
            num_to_keep=3,
            checkpoint_score_attribute='rmse',
            checkpoint_score_order='min',
            checkpoint_frequency=5,
            checkpoint_at_end=True,
        ),
    ), 
)

[I 2023-07-19 02:32:25,666] A new study created in memory with name: optuna


In [3]:
results = tuner.fit()

2023-07-19 02:32:27,846	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-07-19 02:32:29,232	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


2023-07-19 02:32:29,267	INFO wandb.py:320 -- Already logged into W&B.


Trial name,date,done,hostname,iterations_since_restore,mae,mape,node_ip,pid,rmse,time_since_restore,time_this_iter_s,time_total_s,timestamp,training_iteration,trial_id
FSR_Trainable_017adcb8,2023-07-19_02-39-23,True,DESKTOP-0P789CI,1,0.446322,0.116572,172.26.215.93,382558,0.781922,0.651536,0.651536,0.651536,1689701963,1,017adcb8
FSR_Trainable_01ccd703,2023-07-19_02-33-19,True,DESKTOP-0P789CI,100,0.346257,0.096689,172.26.215.93,374463,0.655106,32.8727,0.25926,32.8727,1689701599,100,01ccd703
FSR_Trainable_063e70f4,2023-07-19_02-44-08,True,DESKTOP-0P789CI,2,0.370457,0.106853,172.26.215.93,388904,0.702904,1.68498,0.658058,1.68498,1689702248,2,063e70f4
FSR_Trainable_0649225d,2023-07-19_02-35-30,True,DESKTOP-0P789CI,32,0.356255,0.0947116,172.26.215.93,378231,0.669828,21.7116,0.633955,21.7116,1689701730,32,0649225d
FSR_Trainable_08791bfe,2023-07-19_02-33-28,True,DESKTOP-0P789CI,1,0.504217,0.124197,172.26.215.93,375888,0.775819,0.676482,0.676482,0.676482,1689701608,1,08791bfe
FSR_Trainable_0bd8e5da,2023-07-19_02-32-55,True,DESKTOP-0P789CI,8,0.395933,0.110779,172.26.215.93,374709,0.719601,4.44098,0.54688,4.44098,1689701575,8,0bd8e5da
FSR_Trainable_0c1880fb,2023-07-19_02-39-56,True,DESKTOP-0P789CI,1,0.387535,0.102735,172.26.215.93,383232,0.728985,1.31956,1.31956,1.31956,1689701996,1,0c1880fb
FSR_Trainable_0c5f1635,2023-07-19_02-49-22,True,DESKTOP-0P789CI,1,0.386592,0.0963793,172.26.215.93,394066,0.765337,1.09936,1.09936,1.09936,1689702562,1,0c5f1635
FSR_Trainable_0e31f13a,2023-07-19_02-33-06,True,DESKTOP-0P789CI,2,0.426689,0.110005,172.26.215.93,375206,0.754306,0.881248,0.406422,0.881248,1689701586,2,0e31f13a
FSR_Trainable_0e42846b,2023-07-19_02-39-47,True,DESKTOP-0P789CI,1,0.397781,0.116592,172.26.215.93,382987,0.731992,0.786042,0.786042,0.786042,1689701987,1,0e42846b


(_WandbLoggingActor pid=374535) wandb: Currently logged in as: seokjin. Use `wandb login --relogin` to force relogin
(_WandbLoggingActor pid=374535) wandb: wandb version 0.15.5 is available!  To upgrade, please run:
(_WandbLoggingActor pid=374535) wandb:  $ pip install wandb --upgrade
(_WandbLoggingActor pid=374535) wandb: Tracking run with wandb version 0.15.4
(_WandbLoggingActor pid=374535) wandb: Run data is saved locally in /home/seokj/ray_results/FSR_Trainable_2023-07-19_02-32-25/FSR_Trainable_01ccd703_1_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleIm_2023-07-19_02-32-29/wandb/run-20230719_023240-01ccd703
(_WandbLoggingActor pid=374535) wandb: Run `wandb offline` to turn off syncing.
(_WandbLoggingActor pid=374535) wandb: Syncing run FSR_Trainable_01ccd703
(_WandbLoggingActor pid=374535) wandb: ⭐️ View project at https://wandb.ai/seokjin/FSR-prediction
(_WandbLoggingActor pid=374535) wandb: 🚀 View run at https://wandb.ai/se